In [1]:
import numpy as np
import json
import pandas as pd
import torch
import pickle
from collections import defaultdict
from tqdm import tqdm
# import warnings
# warnings.filterwarnings("ignore")

In [2]:
n_users = 6611
n_items = 79937
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [3]:
pip install dgl-cu110 dglgo -f https://data.dgl.ai/wheels/repo.html

Looking in links: https://data.dgl.ai/wheels/repo.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 221.1/221.1 MB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.2/77.2 KB 592.9 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.1/11.1 MB 23.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.7/51.7 KB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.5/109.5 KB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 71.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 546.8/546.8 KB 39.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.7/84.7 KB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.0/94.0 KB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.5/100.5 KB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.2/121.2 KB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.6/90.6 KB

In [4]:
# load data
# to do: str_id -> int
cite_file = '../input/cs245-2-project-11-graph-based-recommendation/paper_file_ann.txt'
coauthor_file = '../input/cs245-2-project-11-graph-based-recommendation/author_file_ann.txt'
author_train_file = '../input/cs245-2-project-11-graph-based-recommendation/bipartite_train_ann.txt'
author_test_file = '../input/cs245-2-project-11-graph-based-recommendation/bipartite_test_ann.txt'

import random as rd

def generate_test(all_user_ratings):
    ratings_test = {}
    for user in all_user_ratings:
        ratings_test[user] = rd.sample(all_user_ratings[user], 1)[0]
    return ratings_test

def load_data(cite_file, coauthor_file, author_train_file, author_test_file, n_users):
    citation, author_train, coauthor = defaultdict(list), defaultdict(list), defaultdict(list)
    user_ratings_train = defaultdict(list)
    test_ratings = defaultdict(list)
    
    train_users = set()

    with open(cite_file, 'r') as f:
        line = f.readlines()
    for l in line:
        src, tgt = l.strip().split(' ')
        src, tgt = int(src), int(tgt)
        citation['src'].append(src)
        citation['tgt'].append(tgt)

    with open(coauthor_file, 'r') as f:
        line = f.readlines()
    for l in line:
        src, tgt = l.strip().split(' ')
        src, tgt = int(src), int(tgt)
        coauthor['src'].append(src)
        coauthor['tgt'].append(tgt)
        
    
    with open(author_train_file, 'r') as f:
        line = f.readlines()
        train_interacts = len(line)
    for l in line:
        src, tgt = l.strip().split(' ')
        src, tgt = int(src), int(tgt)
        user_ratings_train[src].append(tgt)
        author_train['src'].append(src)
        author_train['tgt'].append(tgt+n_users)
        train_users.add(src)
    
    with open(author_test_file, 'r') as f:
        line = f.readlines()
    for l in line:
        src, tgt = l.strip().split(' ')
        src, tgt = int(src), int(tgt)
        test_ratings[src].append(tgt)
        
    user_ratings_test = generate_test(user_ratings_train)
            
    return citation, author_train, coauthor, user_ratings_train, user_ratings_test, train_interacts, list(train_users), test_ratings

citation, author_train, coauthor, user_ratings_train, user_ratings_test, train_interacts, train_users, test_ratings = \
            load_data(cite_file, coauthor_file, author_train_file, author_test_file, n_users)

In [5]:
# load pickle 
feature_file = '../input/cs245-2-project-11-graph-based-recommendation/feature.pkl'

def load_item_feature(feature_file):
    with open(feature_file, 'rb') as f:
        feature_matrix = pickle.load(f)
    # feature_matrix_shape: 79937*512
    print(feature_matrix.shape)
    return feature_matrix

item_feature = load_item_feature(feature_file)
item_feature_dim = 512

torch.Size([79937, 512])


In [6]:
import scipy.sparse as sp

def create_adj_mat(adj_mat):    
    def mean_adj_single(adj):
        # D^-1 * A
        rowsum = np.array(adj.sum(1))

        d_inv = np.power(rowsum, -1).flatten()
        d_inv[np.isinf(d_inv)] = 0.
        d_mat_inv = sp.diags(d_inv)

        norm_adj = d_mat_inv.dot(adj)
        # norm_adj = adj.dot(d_mat_inv)
        print('generate single-normalized adjacency matrix.')
        return norm_adj.tocoo()

    def normalized_adj_single(adj):
        # D^-1/2 * A * D^-1/2
        rowsum = np.array(adj.sum(1))

        d_inv_sqrt = np.power(rowsum, -0.5).flatten()
        d_inv_sqrt[np.isinf(d_inv_sqrt)] = 0.
        d_mat_inv_sqrt = sp.diags(d_inv_sqrt)

        # bi_lap = adj.dot(d_mat_inv_sqrt).transpose().dot(d_mat_inv_sqrt)
        bi_lap = d_mat_inv_sqrt.dot(adj).dot(d_mat_inv_sqrt)
        return bi_lap.tocoo()

    def check_adj_if_equal(adj):
        dense_A = np.array(adj.todense())
        degree = np.sum(dense_A, axis=1, keepdims=False)

        temp = np.dot(np.diag(np.power(degree, -1)), dense_A)
        print('check normalized adjacency matrix whether equal to this laplacian matrix.')
        return temp

    norm_adj_mat = mean_adj_single(adj_mat + sp.eye(adj_mat.shape[0]))
    # norm_adj_mat = normalized_adj_single(adj_mat + sp.eye(adj_mat.shape[0]))
    mean_adj_mat = mean_adj_single(adj_mat)

    print('already normalize adjacency matrix')
    return adj_mat.tocsr(), norm_adj_mat.tocsr(), mean_adj_mat.tocsr()

def normalize_adj(mx):
    """Row-normalize sparse matrix"""
    rowsum = np.array(mx.sum(1))
    r_inv_sqrt = np.power(rowsum, -0.5).flatten()
    r_inv_sqrt[np.isinf(r_inv_sqrt)] = 0.
    r_mat_inv_sqrt = sp.diags(r_inv_sqrt)
    return mx.dot(r_mat_inv_sqrt).transpose().dot(r_mat_inv_sqrt)

# info graph          
R = sp.dok_matrix((n_users, n_items), dtype=np.float32)
for user in train_users:
    for item in user_ratings_train[user]:
        R[user, item] = 1

          
adj_mat = sp.dok_matrix((n_users + n_items, n_users + n_items), dtype=np.float32)
adj_mat = adj_mat.tolil()
R = R.tolil()

adj_mat[:n_users, n_users:] = R
adj_mat[n_users:, :n_users] = R.T
adj_mat = adj_mat.todok()

_, info_norm_adj,_ = create_adj_mat(adj_mat)

generate single-normalized adjacency matrix.


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:8: RuntimeWarning: divide by zero encountered in power
  


generate single-normalized adjacency matrix.
already normalize adjacency matrix


In [7]:
def build_graph(citation, coauthor, author_train, n_users, n_items, device):
    # build social graph
    soc_src, soc_tgt = coauthor['src'], coauthor['tgt']
    soc_src, soc_tgt = torch.tensor(soc_src).to(device), torch.tensor(soc_tgt).to(device)
    soc_graph = dgl.DGLGraph((soc_src, soc_tgt), num_nodes=n_users)
    soc_graph = dgl.add_reverse_edges(soc_graph)
    
    # build cite graph
    cite_src, cite_tgt = citation['src'], citation['tgt']
    cite_src, cite_tgt = torch.tensor(cite_src).to(device), torch.tensor(cite_tgt).to(device)
    cite_graph = dgl.DGLGraph((cite_src, cite_tgt), num_nodes=n_items)
   
    return soc_graph, cite_graph

In [8]:
def generate_train_batch(user_ratings_train, n, batch_size, train_users, test_ratings):
    t = []
    for b in range(batch_size):
        u = rd.sample(train_users, 1)[0]
        i = rd.sample(user_ratings_train[u], 1)[0]
        j = rd.randint(0, n - 1)
        # one negative sample
        while j in user_ratings_train[u] and j in test_ratings[u]:
            j = rd.randint(0, n - 1)
        t.append([u, i, 1])
        t.append([u, j, 0])
    train_batch = np.asarray(t)
    return train_batch

def generate_test_batch(user_ratings, user_ratings_test, n, train_users, test_ratings):
    t = []
    for u in train_users:
        i = user_ratings_test[u]
        rated = user_ratings[u]
        for j in range(10):
            k = np.random.randint(0, n)
            while k in rated and k in test_ratings[u]:
                k = np.random.randint(0, n)
            t.append([u, i, 1])
            t.append([u, k, 0])
    test_batch = np.asarray(t)
    return test_batch

In [9]:
from sklearn.metrics import precision_score, recall_score, f1_score
import numpy as np

def evaluation(pred, labels):
    auc = cal_auc(pred, labels)
    pred = (pred > 0.5).astype(int)
    labels = (labels > 0.5).astype(int)
    precision = precision_score(pred, labels, average='binary')
    recall = recall_score(pred, labels, average='binary')
    f1score = f1_score(pred, labels, average='binary')
    return auc, precision, recall, f1score

def cal_auc(pred, labels):
    P_ind = []  # 正样本下标
    F_ind = []  # 负样本下标

    #  计数过程
    for ind, val in enumerate(labels):
        if val > 0.5:
            P_ind.append(ind)
        else:
            F_ind.append(ind)

    new_data = [[p, l] for p, l in zip(pred, labels)]
    new_data.sort(key=lambda x:x[0])

    # 求正样本rank之和
    rank_sum = 0
    for ind, [prob,label] in enumerate(new_data):
        if label>0.5:
            rank_sum+=ind
    auc = (rank_sum - len(P_ind)*(1+len(P_ind))/2) / (len(P_ind)*len(F_ind))
    return auc

In [10]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.sparse as sparse


import dgl
import dgl.function as fn

DGL backend not selected or invalid.  Assuming PyTorch for now.


Setting the default backend to "pytorch". You can change it in the ~/.dgl/config.json file or export the DGLBACKEND environment variable.  Valid options are: pytorch, mxnet, tensorflow (all lowercase)


In [11]:
class GATLayer(nn.Module):
    def __init__(self, g, in_dim, out_dim):
        super(GATLayer, self).__init__()
        self.g = g
        # equation (1)
        self.fc = nn.Linear(in_dim, out_dim, bias=False)
        # equation (2)
        self.attn_fc = nn.Linear(2 * out_dim, 1, bias=False)

    def edge_attention(self, edges):
        # edge UDF for equation (2)
        z2 = torch.cat([edges.src['z'], edges.dst['z']], dim=1)
        a = self.attn_fc(z2)
        return {'e': F.leaky_relu(a)}

    def message_func(self, edges):
        # message UDF for equation (3) & (4)
        return {'z': edges.src['z'], 'e': edges.data['e']}

    def reduce_func(self, nodes):
        # reduce UDF for equation (3) & (4)
        # equation (3)
        alpha = F.softmax(nodes.mailbox['e'], dim=1)
        # equation (4)
        h = torch.sum(alpha * nodes.mailbox['z'], dim=1)
        return {'h': h}

    def forward(self, h):
        # equation (1)
        z = self.fc(h)
        self.g.ndata['z'] = z
        # equation (2)
        self.g.apply_edges(self.edge_attention)
        # equation (3) & (4)
        self.g.update_all(self.message_func, self.reduce_func)
        return self.g.ndata.pop('h')

    
class MultiHeadGATLayer(nn.Module):
    def __init__(self, g, in_dim, out_dim, num_heads, merge='cat'):
        super(MultiHeadGATLayer, self).__init__()
        self.heads = nn.ModuleList()
        for i in range(num_heads):
            self.heads.append(GATLayer(g, in_dim, out_dim))
        self.merge = merge

    def forward(self, h):
        head_outs = [attn_head(h) for attn_head in self.heads]
        if self.merge == 'cat':
            # concat on the output feature dimension (dim=1)
            return torch.cat(head_outs, dim=1)
        else:
            # merge using average
            return torch.mean(torch.stack(head_outs))

class GAT(nn.Module):
    def __init__(self, g, in_dim, hidden_dim, out_dim, num_heads):
        super(GAT, self).__init__()
        self.layer1 = MultiHeadGATLayer(g, in_dim, hidden_dim, num_heads)
        # Be aware that the input dimension is hidden_dim*num_heads since
        # multiple head outputs are concatenated together. Also, only
        # one attention head in the output layer.
        self.layer2 = MultiHeadGATLayer(g, hidden_dim * num_heads, out_dim, 1)

    def forward(self, h):
        h = self.layer1(h)
        h = F.elu(h)
        h = self.layer2(h)
        return h

In [12]:
class LightGCNLayer(nn.Module):
    def __init__(self, n_users, n_items, n_layers, emb_dim, graph, device, keep_prob=0.6):
        super(LightGCNLayer, self).__init__()
        self.n_users = n_users
        self.n_items = n_items
        self.n_layers = n_layers
        self.graph = graph
        self.graph = self._convert_sp_mat_to_sp_tensor(self.graph)
        self.graph = self.graph.coalesce().to(device)
        self.keep_prob = keep_prob
    
    def _convert_sp_mat_to_sp_tensor(self, X):
        coo = X.tocoo().astype(np.float32)
        row = torch.Tensor(coo.row).long()
        col = torch.Tensor(coo.col).long()
        index = torch.stack([row, col])
        data = torch.FloatTensor(coo.data)
        return torch.sparse.FloatTensor(index, data, torch.Size(coo.shape))
    
    def __dropout_x(self, x, keep_prob):
        size = x.size()
        index = x.indices().t()
        values = x.values()
        random_index = torch.rand(len(values)) + keep_prob
        random_index = random_index.int().bool()
        index = index[random_index]
        values = values[random_index]/keep_prob
        g = torch.sparse.FloatTensor(index.t(), values, size)
        return g
    
    def __dropout(self, keep_prob):
        graph = self.__dropout_x(self.graph, keep_prob)
        return graph
    
    def forward(self, user_embedding, item_embedding):
        all_emb = torch.cat([user_embedding, item_embedding])
        embs = [all_emb]
        if self.training:
            g_droped = self.__dropout(self.keep_prob)
        else:
            g_droped = self.graph
        for layer in range(self.n_layers):
            all_emb = torch.sparse.mm(g_droped, all_emb)
            embs.append(all_emb)
        embs = torch.stack(embs, dim=1)
        light_out = torch.mean(embs, dim=1)
        users, items = torch.split(light_out, [self.n_users, self.n_items])
        return users, items

In [13]:
# class NGCFLayer(nn.Module):
#     def __init__(self, norm_adj, emb_dim, n_users, device):
#         super(NGCFLayer, self).__init__()
#         self.n_users = n_users
#         self.norm_adj = norm_adj
#         self.sparse_norm_adj = self._convert_sp_mat_to_sp_tensor(self.norm_adj).to(device)
#         self.layers = [8]
#         self.node_dropout = 0.1
#         self.mess_dropout = [0.1,0.1,0.1]
#         self.emb_size = emb_dim
#         self.init_weight()
    
#     def init_weight(self):
#         # xavier init
#         initializer = nn.init.xavier_uniform_
#         self.weight_dict = nn.ParameterDict()
#         layers = [self.emb_size] + self.layers
#         for k in range(len(self.layers)):
#             self.weight_dict.update({'W_gc_%d'%k: nn.Parameter(initializer(torch.empty(layers[k],
#                                                                       layers[k+1])))})
#             self.weight_dict.update({'b_gc_%d'%k: nn.Parameter(initializer(torch.empty(1, layers[k+1])))})

#             self.weight_dict.update({'W_bi_%d'%k: nn.Parameter(initializer(torch.empty(layers[k],
#                                                                       layers[k+1])))})
#             self.weight_dict.update({'b_bi_%d'%k: nn.Parameter(initializer(torch.empty(1, layers[k+1])))})
    
    
#     def sparse_dropout(self, x, rate, noise_shape):
#         random_tensor = 1 - rate
#         random_tensor += torch.rand(noise_shape).to(x.device)
#         dropout_mask = torch.floor(random_tensor).type(torch.bool)
#         i = x._indices()
#         v = x._values()

#         i = i[:, dropout_mask]
#         v = v[dropout_mask]

#         out = torch.sparse.FloatTensor(i, v, x.shape).to(x.device)
#         return out * (1. / (1 - rate))
    
    
#     def _convert_sp_mat_to_sp_tensor(self, X):
#         coo = X.tocoo()
#         i = torch.LongTensor([coo.row, coo.col])
#         v = torch.from_numpy(coo.data).float()
#         return torch.sparse.FloatTensor(i, v, coo.shape)
    
#     def forward(self, user_embedding, item_embedding, drop_flag=True):

#         A_hat = self.sparse_dropout(self.sparse_norm_adj,
#                                     self.node_dropout,
#                                     self.sparse_norm_adj._nnz()) if drop_flag else self.sparse_norm_adj

#         ego_embeddings = torch.cat([user_embedding, item_embedding], 0)

# #         all_embeddings = [ego_embeddings]

#         for k in range(len(self.layers)):
#             side_embeddings = torch.sparse.mm(A_hat, ego_embeddings)

#             # transformed sum messages of neighbors.
#             sum_embeddings = torch.matmul(side_embeddings, self.weight_dict['W_gc_%d' % k]) \
#                                              + self.weight_dict['b_gc_%d' % k]

#             # bi messages of neighbors.
#             # element-wise product
#             bi_embeddings = torch.mul(ego_embeddings, side_embeddings)
#             # transformed bi messages of neighbors.
#             bi_embeddings = torch.matmul(bi_embeddings, self.weight_dict['W_bi_%d' % k]) \
#                                             + self.weight_dict['b_bi_%d' % k]

#             # non-linear activation.
#             ego_embeddings = nn.LeakyReLU(negative_slope=0.2)(sum_embeddings + bi_embeddings)

#             # message dropout.soc_norm_adj
#             ego_embeddings = nn.Dropout(self.mess_dropout[k])(ego_embeddings)

#             # normalize the distribution of embeddings.
#             norm_embeddings = F.normalize(ego_embeddings, p=2, dim=1)
# #             all_embeddings += [norm_embeddings]
#             all_embeddings = norm_embeddings

# #         all_embeddings = torch.cat(all_embeddings, 1)
#         u_g_embeddings = all_embeddings[:self.n_users, :]
#         i_g_embeddings = all_embeddings[self.n_users:, :]
#         return u_g_embeddings, i_g_embeddings

In [14]:
class OurModel(nn.Module):
    def __init__(self, soc_graph, info_norm_adj, cite_graph, n_users, n_items, item_feature, emb_dim, item_feature_dim, num_layers, lightgcn_n_layers, device):
        super(OurModel, self).__init__()
        self.item_feature = torch.tensor(item_feature).to(device)
        
        self.n_users = n_users
        self.n_items = n_items
        self.emb_dim = emb_dim
#         self.num_layers = num_layers
        self.num_layers = num_layers
        self.lightgcn_n_layers = lightgcn_n_layers
        
        self.emb_dim = emb_dim

        self.user_embedding = nn.Embedding(n_users, self.emb_dim, sparse=True).to(device)
        self.item_embedding = nn.Embedding(n_items, self.emb_dim, sparse=True).to(device)
          
        self.info_norm_adj = info_norm_adj
#         self.cite_edges = cite_edges
#         self.soc_edges = soc_edges
        self.soc_graph = soc_graph
        self.cite_graph = cite_graph
        
        self.hidden_dim = self.emb_dim
        self.dim = self.emb_dim
        self.num_heads = 1
        
#         self.soc_gat_layer = nn.ModuleList([SpGAT(nfeat=self.emb_dim, nhid=self.hidden_dim, nclass=2, dropout=self.dropout, nheads=self.num_heads, alpha=self.alpha)])
#         self.cite_gat_layer = nn.ModuleList([SpGAT(nfeat=self.emb_dim, nhid=self.hidden_dim, nclass=2, dropout=self.dropout, nheads=self.num_heads, alpha=self.alpha)])
#         self.info_ngcf_layer = nn.ModuleList([NGCFLayer(self.info_norm_adj, self.emb_dim, self.n_users, device)])
        self.info_lightgcn_layer = nn.ModuleList([LightGCNLayer(n_users=self.n_users, n_items=self.n_items, n_layers=self.lightgcn_n_layers, emb_dim=self.emb_dim, graph=self.info_norm_adj, device=device) for i in range(self.num_layers)])
        self.soc_gat_layer = nn.ModuleList([GAT(self.soc_graph, self.dim, self.hidden_dim, self.emb_dim, self.num_heads) for i in range(self.num_layers)])
        self.cite_gat_layer = nn.ModuleList([GAT(self.cite_graph, self.dim, self.hidden_dim, self.emb_dim, self.num_heads) for i in range(self.num_layers)])
        
        self.reduce_dim_layer = nn.Linear(item_feature_dim, self.emb_dim)
        
        self.initializer = nn.init.xavier_uniform_
        self.loss = nn.BCELoss()
        
    
    def _convert_distribution(self, x):
        mean, std = torch.mean(x), torch.std(x)
        y = (x-mean)*0.2/std
        return y
    
    def init_nodes_feature(self):
        first_item_feature = self._convert_distribution(self.item_feature)
        second_item_feature = self.reduce_dim_layer(first_item_feature)
        third_item_feature = self._convert_distribution(second_item_feature)
        
        self.init_user_feature = self.user_embedding.weight
        self.init_item_feature = third_item_feature + self.item_embedding.weight
    
    def forward(self, users, items):
        self.init_nodes_feature()
        
        item_embedding = self.init_item_feature
        user_embedding = self.init_user_feature
        
        for i in range(self.num_layers):
            user_embedding_from_consumed_items, item_embedding_from_related_users = self.info_lightgcn_layer[i](user_embedding, item_embedding)
            gcn_user_embedding = self.soc_gat_layer[i](user_embedding)[:self.n_users]
            gcn_item_embedding = self.cite_gat_layer[i](item_embedding)[:self.n_items]
        
            item_embedding = item_embedding_from_related_users + gcn_item_embedding
            user_embedding = user_embedding_from_consumed_items + gcn_user_embedding
#         item_embedding = torch.matmul(item_embedding_from_related_users, self.item_fusion_weight_1)+torch.matmul(gcn_item_embedding, self.item_fusion_weight_2)
#         user_embedding = torch.matmul(user_embedding_from_consumed_items, self.user_fusion_weight_1)+torch.matmul(gcn_user_embedding, self.user_fusion_weight_2)
        
        final_item_embedding = torch.index_select(item_embedding, 0, items)
        final_user_embedding = torch.index_select(user_embedding, 0, users)
        
        # prediction layer
        pred = torch.sigmoid(torch.sum(final_user_embedding*final_item_embedding, axis=1, keepdim=True))
        return pred

    def cal_loss(self, users, items, labels):
        pred = self.forward(users, items)
        pred = pred.to(torch.float32)
        labels = labels.to(torch.float32)
        labels = labels.reshape(-1, 1)
        loss = self.loss(pred, labels)
        return loss

In [15]:
# params
batch_size = 65536
emb_dim = 64
lr = 0.001
num_epoches = 350

In [16]:
############################# CREATE MODEL ##############################
soc_graph, cite_graph = build_graph(citation, coauthor, author_train, n_users, n_items, device)
model = OurModel(soc_graph, info_norm_adj, cite_graph, n_users, n_items, item_feature, emb_dim, item_feature_dim, 2, 3, device)
model = model.to(device)
opt = torch.optim.Adam(lr=lr, params=model.parameters(), weight_decay=0.0001)

/opt/conda/lib/python3.7/site-packages/dgl/heterograph.py:72: DGLWarning: Recommend creating graphs by `dgl.graph(data)` instead of `dgl.DGLGraph(data)`.
  dgl_warning('Recommend creating graphs by `dgl.graph(data)`'
/opt/conda/lib/python3.7/site-packages/dgl/heterograph.py:84: DGLWarning: Keyword arguments ['num_nodes'] are deprecated in v0.5, and can be safely removed in all cases.
  ' removed in all cases.'.format(list(deprecate_kwargs.keys())))
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:4: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  after removing the cwd from sys.path.


In [17]:
########################### START TRAINING & TESTING & EVALUATION#####################################
all_loss = []
performance = []
best_auc = 0
best_precision = 0
model_save_path = './model'
for epoch in range(1, num_epoches+1):
    model.train()
    train_loss = 0
    n_batches = train_interacts // batch_size + 1
    pbar = tqdm(total=n_batches*batch_size*2)
    for i in range(n_batches):
        uij = generate_train_batch(user_ratings_train, n_items, batch_size, train_users, test_ratings)
        users, items, labels = uij[:, 0], uij[:, 1], uij[:, 2]
        users, items, labels = torch.tensor(users).to(device), torch.tensor(items).to(device), torch.tensor(labels).to(device)
        loss = model.cal_loss(users, items, labels)
        train_loss += loss.item()
        opt.zero_grad()
        loss.backward()
        opt.step()
        pbar.update(batch_size*2)
    train_loss /= n_batches
    all_loss.append(train_loss)
    print("epoch {}, train loss: {:4f}".format(epoch, train_loss))
    
    model.eval()
#     auc_list, precision_list, recall_list, f1score_list = [], [], [], []
    with torch.no_grad():
        t_uij = generate_test_batch(user_ratings_train, user_ratings_test, n_items, train_users, test_ratings)
        users, items, labels = t_uij[:, 0], t_uij[:, 1], t_uij[:, 2]
        users, items = torch.tensor(users).to(device), torch.tensor(items).to(device)
        pred = model(users, items)
        pred = pred.cpu().numpy()
        auc, precision, recall, f1score = evaluation(pred, labels)
#         auc_list.append(auc)
#         precision_list.append(precision)
#         recall_list.append(recall)
#         f1score_list.append(f1score)
#         auc, precision, recall, f1score = np.mean(auc_list), np.mean(precision_list), np.mean(recall_list), np.mean(f1score_list)
        performance.append([auc, precision, recall, f1score])
        print("Evaluation: auc:{}, precision:{}, recall:{}, f1:{}".format(auc, precision, recall, f1score)) 
        if auc > best_auc:
            best_auc = auc
            best_precision = precision
            state = {'net': model.state_dict(), 'opt':opt.state_dict(), 'epoch':epoch}
            torch.save(state, model_save_path)
    
all_loss = np.array(all_loss)
performance = np.array(performance)
np.save("./loss.npy", all_loss)
np.save("./performance.npy", performance)

100%|██████████| 2752512/2752512 [01:15<00:00, 37358.91it/s]

epoch 1, train loss: 0.627365
Evaluation: auc:0.8776444240610375, precision:0.9875832828588734, recall:0.6661355558279201, f1:0.7956181228194307



100%|██████████| 2752512/2752512 [01:20<00:00, 34307.27it/s]

100%|██████████| 2752512/2752512 [01:12<00:00, 39247.04it/s]

epoch 2, train loss: 0.560492
Evaluation: auc:0.9158146472358033, precision:0.9910660205935796, recall:0.6828235195927054, f1:0.8085637338472561


100%|██████████| 2752512/2752512 [01:13<00:00, 37158.65it/s]

...

epoch 348, train loss: 0.104487
Evaluation: auc:0.9952276729849318, precision:0.975923682616596, recall:0.9674126776842137, f1:0.9716495428196682


100%|██████████| 2752512/2752512 [01:13<00:00, 39329.16it/s]

epoch 349, train loss: 0.103872
Evaluation: auc:0.9953741120499993, precision:0.983343428225318, recall:0.9622451398767188, f1:0.9726798873644478



100%|██████████| 2752512/2752512 [01:17<00:00, 35328.23it/s]

100%|██████████| 2752512/2752512 [01:13<00:00, 36295.51it/s]

epoch 350, train loss: 0.103729
Evaluation: auc:0.9953813860586301, precision:0.988037552998183, recall:0.9547290179093995, f1:0.971097749732111


In [18]:
# from collections import OrderedDict

ckpt = torch.load('./model')

# model_weights = OrderedDict()
# for k, v in ckpt['net'].items():
#     if not k == 'init_user_feature':
#         model_weights[k] = v
# print(model_weights)

# model.load_state_dict(model_weights)
model.load_state_dict(ckpt['net'])
opt.load_state_dict(ckpt['opt'])
print("##################### Best epoch : {} ###########################".format(ckpt['epoch']))

##################### Best epoch : 340 ###########################


In [19]:
users, items = [], []
with open(author_test_file, 'r') as f:
    line = f.readlines()
for l in line:
    src, tgt = l.strip().split(' ')
    src, tgt = int(src), int(tgt)
    users.append(src)
    items.append(tgt)
users = np.array(users)
items = np.array(items)

model.eval()
with torch.no_grad():
    users, items = torch.tensor(users).to(device), torch.tensor(items).to(device)
    pred = model(users, items)
    pred = pred.cpu().numpy()
    pred = pred.reshape(-1)

import pandas as pd

data = []
for index, p in enumerate(list(pred)):
    tp = [index, p]
    data.append(tp)

df = pd.DataFrame(data, columns=["Index", "Probability"])
df.to_csv('./submission.csv', index=False)